In [15]:
import sys
import os 

paths = ['../src/', '../src/api/v6']
for path in paths:
    path = os.path.normcase(path)
    if not any(os.path.normcase(sp) == path for sp in sys.path):
        sys.path.append(path)

In [16]:
sys.path

['/Users/asaf/Workspace/biu/complex-utterance-to-code/notebooks',
 '/Users/asaf/Workspace/biu/complex-utterance-to-code',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python39.zip',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python3.9',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python3.9/lib-dynload',
 '',
 '/Users/asaf/.local/lib/python3.9/site-packages',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python3.9/site-packages',
 '../src/',
 '../src/api/v6']

In [3]:
text = """
    [ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ 
						[ recipient1, recipient2 ]
					]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]
 """

In [4]:
import ast
from representations.tree.tree import Tree
from representations.builders.ast.tearers.tearer_factory import TearerFactory

In [5]:
tree = Tree.unparse(text)
print(tree)

	[ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ [ recipient1, recipient2 ] ]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]



In [6]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

source = Location.resolve_from_text('Walmart')
destination = Location.resolve_from_text('Walgreens')
directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time)
Responder.respond(response=directions)
recipient1 = Contact.resolve_from_text('Pepsi')
recipient2 = Contact.resolve_from_text('Walgreens')
for recipient in [recipient1, recipient2]:
    exact_content = Content.resolve_from_text('check it out')
    Message.send_message(recipient=recipient, exact_content=exact_content)


In [8]:
import pandas as pd

df = pd.read_csv('../build/eval_complex_utterance_to_code_with_intermediate_61_20230509.csv.gz')

In [9]:
df.iloc[0]['text']

'Check the availability of Pepsi at Walmart and also check it at Walgreens.'

In [10]:
df.iloc[0]['code_rep']

"[ Module [ product_name = ProductName.resolve_from_text('Pepsi') ] [ location = Location.resolve_from_text('Walmart') ] [ products = Shopping.find_products(product_name=product_name, location=location) ] [ Responder.respond(response=products) ] [ location = Location.resolve_from_text('Walgreens') ] [ products = Shopping.find_products(product_name=product_name, location=location) ] [ Responder.respond(response=products) ] ]"

In [11]:
tree = Tree.unparse(df.iloc[0]['code_rep'])

In [12]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

product_name = ProductName.resolve_from_text('Pepsi')
location = Location.resolve_from_text('Walmart')
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)
location = Location.resolve_from_text('Walgreens')
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)


In [17]:
import utils.api_utils

In [27]:
text = df.iloc[34]['code_rep']

In [28]:
tree = Tree.unparse(text)
print(tree)

	[ Module
		[ locations = Location.resolve_many_from_text('In N Out') ]
		[ locations = utils.sort(locations, 'nearest') ]
		[ location = utils.first(locations) ]
		[ map_entities = Map.find_on_map(location=location) ]
		[ Responder.respond(response=map_entities) ]
		[ recipient = Contact.resolve_from_text('Bryan') ]
		[ content = Content.resolve_from_text('his order') ]
		[ Messages.send_message(recipient=recipient, content=content) ]
	]



In [29]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

locations = Location.resolve_many_from_text('In N Out')
locations = utils.sort(locations, 'nearest')
location = utils.first(locations)
map_entities = Map.find_on_map(location=location)
Responder.respond(response=map_entities)
recipient = Contact.resolve_from_text('Bryan')
content = Content.resolve_from_text('his order')
Messages.send_message(recipient=recipient, content=content)
